In [33]:
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [28]:
## Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")

# Import data that will be used to look for adresses

In [40]:
file_loc = 'inscricao_imoveis.xlsx'
imoveis = pd.read_excel(file_loc)
imoveis_inscricao = imoveis['IMOVEL']
teste = imoveis_inscricao.head()

In [ ]:
type(teste[4])

# Create driver

In [41]:
#driver = webdriver.Chrome(ChromeDriverManager().install())
# Create service
webdriver_service = Service(ChromeDriverManager().install())
# Create driver
driver = webdriver.Chrome(service = webdriver_service, options = chrome_options)

In [42]:
page_url = 'https://iptu.campogrande.ms.gov.br/Index'
driver.get(page_url)

In [12]:
inscricao = driver.find_element(By.ID, "inputInscricao")
inscricao.send_keys("0")
inscricao.send_keys(Keys.ENTER)
time.sleep(2)
invalid = driver.find_element(By.CLASS_NAME, 'toast-message')

In [ ]:
invalid.text

# Select values for adress

In [ ]:
endereco_list = []
bairro_list = []

for imovel in imoveis_inscricao:
    driver.get(page_url)
    time.sleep(2)
    inscricao = driver.find_element(By.ID, "inputInscricao")
    inscricao.send_keys(imovel)
    inscricao.send_keys(Keys.ENTER)
    time.sleep(1)
    try:
        invalid = driver.find_element(By.CLASS_NAME, 'toast-message')
        if invalid != 'Inscrição Municipal inválida! Por favor, verifique se a informação está correta e tente novamente.':
            endereco = driver.find_element(By.XPATH, '/html/body/section[1]/div/div/div[1]/div[4]')
            bairro = driver.find_element(By.XPATH, '/html/body/section[1]/div/div/div[2]/div[2]')
            endereco_list.append(endereco.text)
            bairro_list.append(bairro.text)
    except:
        continue

In [ ]:
#print(endereco_list)
#print(bairro_list)
len(endereco_list)

# Write in the xlsx file and save

In [38]:
lend = endereco_list
lbair = bairro_list

In [39]:
df = pd.DataFrame({'INSCRICAO': imoveis_inscricao, 'ENDERECO': lend, 'BAIRRO': lbair})
df.to_excel('enredecos_inscricao.xlsx', sheet_name='sheet1', index=False)